# Tensorflow ImageNet FGM demo for Securing AI Lab deployment

This notebook contains a demonstration of the Fast Gradient Method (FGM) attack and Spatial Smoothing defense on the DGX workstation.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `localhost:30080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `localhost:35000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward 30080 localhost:30080
    LocalForward 35000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:30080"

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_fgm_defense"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [2]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
initdb              Initialize the RESTful API database 
services            Launch the Minio S3 and MLFlow Tracking services 
teardown            Destroy service containers 
workflows           Create workflows tarball 


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both train and val-sorted contain a full set of labeled training and test images. val-sorted-# directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

Each of the directories listed follows the general subfolder structure below:

    ImageNet
    │ 
    │ 
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    │ 
    └── val-sorted
    │ 
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}

The subfolders under each directory are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [3]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [4]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"howard_imagenet_fgm_defense"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

# FGM Model Setup and Attack

For this section of the demo we will first start by loading in the respective baseline ResNet50 model and evaluating its performance on a subset of the ImageNet validation data.

In [5]:
response_init_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P model_architecture=resnet50",
        "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Model initialization and ImageNet evaluation job submitted.")
print("")
pprint.pprint(response_init_model)


Model initialization and ImageNet evaluation job submitted.

{'createdOn': '2020-10-29T08:04:24.264119',
 'dependsOn': None,
 'entryPoint': 'init_model',
 'entryPointKwargs': '-P batch_size=20 -P model_architecture=resnet50 -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000',
 'experimentId': 8,
 'jobId': 'aba2366c-656f-402c-aa67-c9f56b035389',
 'lastModified': '2020-10-29T08:04:24.264119',
 'mlflowRunId': None,
 'queue': 'tensorflow_gpu',
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/5d50af6cba51463782a986697f970d12/workflows.tar.gz'}


Now that we have our baseline model loaded and stored, next we will apply the fast-gradient method (FGM) evasion attack on the model to generate adversarial images.
Then, after we have the adversarial images, we will reapply the attack with spatial smoothing defense enabled.

We will then test the model under both sets of images (FGM attack, FGM attack + preprocessing defense) and apply standard machine learning accuracy metrics on the inference results. 

This will give us a general sense of the effectivness and robustness of the FGM evasion attack with regards to a particular preprocessing defense.


In [6]:
def mlflow_run_id_is_not_known(response):
    return response["mlflowRunId"] is None and response["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P eps_step=10"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_init_model['jobId']
)

print("FGM attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack)
print("")


FGM attack (ResNet50 architecture) job submitted

{'createdOn': '2020-10-29T08:04:24.311030',
 'dependsOn': 'aba2366c-656f-402c-aa67-c9f56b035389',
 'entryPoint': 'fgm',
 'entryPointKwargs': '-P '
                     'model=howard_imagenet_fgm_defense_default_pretrained_resnet50/1 '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P eps_step=10',
 'experimentId': 8,
 'jobId': '52c9c675-b9f2-4b27-9f85-b124c8cf4a99',
 'lastModified': '2020-10-29T08:04:24.311030',
 'mlflowRunId': None,
 'queue': 'tensorflow_gpu',
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/7241595cd73a47c7826375645c0353c9/workflows.tar.gz'}



In [7]:
# Wait for FGM attack to complete.Then evaluate results.
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack):
    time.sleep(1)
    response_fgm_resnet50_attack = restapi_client.get_job_by_id(response_fgm_resnet50_attack["jobId"])
    
    
response_evaluate_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P batch_size=20",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2020-10-29T08:05:08.983854',
 'dependsOn': '52c9c675-b9f2-4b27-9f85-b124c8cf4a99',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=d93acec4ed7c47c3bdd048f8ea3aab19 -P '
                     'model=howard_imagenet_fgm_defense_default_pretrained_resnet50/1 '
                     '-P batch_size=20 -P '
                     'dataset_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'dataset_name=adv_testing',
 'experimentId': 8,
 'jobId': 'e3e6412a-1e1f-48e5-9fa4-c50b44ab56d1',
 'lastModified': '2020-10-29T08:05:08.983854',
 'mlflowRunId': None,
 'queue': 'tensorflow_gpu',
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/33fb2da74f7f47a9b37aa3d0316230d9/workflows.tar.gz'}



Now we will repeat the procedure for FGM attack with spatial smoothing defense enabled.

In [11]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P apply_spatial_smoothing=true",
            
        ]
    ),
    queue="tensorflow_gpu",
)

print("FGM attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

FGM attack (ResNet50 architecture) job submitted

{'createdOn': '2020-10-29T08:12:04.975711',
 'dependsOn': None,
 'entryPoint': 'fgm',
 'entryPointKwargs': '-P '
                     'model=howard_imagenet_fgm_defense_default_pretrained_resnet50/None '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P apply_spatial_smoothing=true',
 'experimentId': 8,
 'jobId': '4f96222e-f4ce-480b-af6a-87d1f2539a94',
 'lastModified': '2020-10-29T08:12:04.975711',
 'mlflowRunId': None,
 'queue': 'tensorflow_gpu',
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/9ab1b95020b64e209bd0efa6af7b6309/workflows.tar.gz'}



In [12]:
# Wait for FGM attack to complete.Then evaluate results.
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model={EXPERIMENT_NAME}_default_pretrained_resnet50/None",
            "-P batch_size=20",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2020-10-29T08:12:36.914342',
 'dependsOn': '4f96222e-f4ce-480b-af6a-87d1f2539a94',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=41d3baa9b4a44b759f4554ca599bd981 -P '
                     'model=howard_imagenet_fgm_defense_default_pretrained_resnet50/None '
                     '-P batch_size=20 -P '
                     'dataset_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'dataset_name=adv_testing',
 'experimentId': 8,
 'jobId': '2f101e8e-1568-4734-a602-1ce5250ee84b',
 'lastModified': '2020-10-29T08:12:36.914342',
 'mlflowRunId': None,
 'queue': 'tensorflow_gpu',
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/157cdb9f42ce434f85b930fe97cb9d7a/workflows.tar.gz'}



## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the FGM attack applied to the LeNet-5 architecture,

In [ ]:
fgm_run = mlflow_client.get_run(response_evaluate_fgm_resnet50_attack_def["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(fgm_run.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(fgm_run.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(fgm_run.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).